Save 1 row with the latest date currency

In [17]:
import requests
import psycopg2

# Set the API endpoint and your API key
api_endpoint = 'https://openexchangerates.org/api/historical/'
api_key = '5319307e72fb4e689d46e34078ad6683'

# Set the date for which you want to get the exchange rate data
date = '2022-12-20'

# Set the currency pair for which you want to get the exchange rate data
base = 'USD'
symbols = 'EUR'

# Construct the API request URL
url = f'{api_endpoint}{date}.json?base={base}&symbols={symbols}&app_id={api_key}'

# Send the request to the API and store the response
response = requests.get(url, verify=False)

# Check the response status code to make sure the request was successful
if response.status_code == 200:
    # Get the exchange rate data from the response
    data = response.json()['rates']

    # Connect to the local database
    conn = psycopg2.connect(host='localhost', port=5432, dbname='postgres', user='postgres', password='admin')

    # Create a cursor to execute database commands
    cursor = conn.cursor()

    # Check if the exchange_rates table exists
    cursor.execute("SELECT to_regclass('exchange_rates')")
    table_exists = cursor.fetchone()[0]

    # If the table does not exist, create it
    if table_exists is None:
        cursor.execute(
            'CREATE TABLE exchange_rates (date DATE NOT NULL, base_currency CHAR(3) NOT NULL, symbol CHAR(3) NOT NULL, rate NUMERIC(10, 6) NOT NULL, PRIMARY KEY (date, base_currency, symbol))'
        )

    # Check if the row already exists in the table
    cursor.execute('SELECT 1 FROM exchange_rates WHERE date=%s AND base_currency=%s AND symbol=%s', (date, base, symbols))
    row_exists = cursor.fetchone()

    # Insert the row if it does not exist
    if not row_exists:
        cursor.execute('INSERT INTO exchange_rates (date, base_currency, symbol, rate) VALUES (%s, %s, %s, %s)',
                    (date, base, symbols, data[symbols]))

    # Commit the transaction
    conn.commit()

    # Close the cursor and connection
    cursor.close()
    conn.close()

    print('Exchange rate data was successfully stored in the database.')
else:
    print(f'Error {response.status_code}: {response.text}')



c:\Users\enizamov\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openexchangerates.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Exchange rate data was successfully stored in the database.


Saving historical data with the loop

In [20]:
import requests
import psycopg2
from datetime import datetime, timedelta

# Set the API endpoint and your API key
api_endpoint = 'https://openexchangerates.org/api/historical/'
api_key = '5319307e72fb4e689d46e34078ad6683'

# Set the date range for which you want to get the exchange rate data
start_date = datetime(2021, 1, 1)
end_date = datetime(2022, 12, 31)

# Set the currency pair for which you want to get the exchange rate data
base = 'USD'
symbols = 'EUR'

# Connect to the local database
conn = psycopg2.connect(host='localhost', port=5432, dbname='postgres', user='postgres', password='admin')

# Create a cursor to execute database commands
cursor = conn.cursor()

# Check if the exchange_rates table exists
cursor.execute("SELECT to_regclass('exchange_rates')")
table_exists = cursor.fetchone()[0]

# If the table does not exist, create it
if table_exists is None:
    cursor.execute(
        """
        CREATE TABLE exchange_rates (
            date DATE NOT NULL, 
            base_currency CHAR(3) NOT NULL, 
            symbol CHAR(3) NOT NULL, 
            rate NUMERIC(10, 6) NOT NULL, 
            PRIMARY KEY (date, base_currency, symbol)
            )
        """
    )

# Loop through each day in the date range
current_date = start_date
while current_date <= end_date:
    # Construct the API request URL
    url = f'{api_endpoint}{current_date.strftime("%Y-%m-%d")}.json?base={base}&symbols={symbols}&app_id={api_key}'

    # Send the request to the API and store the response
    response = requests.get(url, verify=False)

    # Check the response status code to make sure the request was successful
    if response.status_code == 200:
        # Get the exchange rate data from the response
        data = response.json()['rates']

        # Insert the exchange rate data into the database
        cursor.execute('INSERT INTO exchange_rates (date, base_currency, symbol, rate) VALUES (%s, %s, %s, %s)',
                    (current_date, base, symbols, data[symbols]))

        # Commit the transaction
        conn.commit()

    # Increment the date by one day
    current_date += timedelta(days=1)


# Close the cursor and connection
cursor.close()
conn.close()

c:\Users\enizamov\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openexchangerates.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\enizamov\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openexchangerates.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\enizamov\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openexchangerates.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/e